In [1]:
import numpy as np
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import pandas as pd

In [2]:
var_names = ["demand", "trans", "storage", "inertia", "solar"]
regression_out = pd.DataFrame(index=pd.RangeIndex(stop=((len(var_names)**2-len(var_names)))//2), columns=["x", "y", "bx", "by", "bxy", "a", "R^2"])

In [3]:
# Create model 
model = Pipeline([("poly", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
                  ("regression", LinearRegression())])

In [13]:
np.loadtxt("../Data/Normalized Data/demand-solar.csv",delimiter=",",skiprows=1)

array([[0.02      , 0.        , 0.17520008],
       [0.02      , 0.01      , 0.15216511],
       [0.02      , 0.02      , 0.15214968],
       [0.02      , 0.03      , 0.15213483],
       [0.02      , 0.04      , 0.15212054],
       [0.015     , 0.        , 0.18634118],
       [0.015     , 0.01      , 0.16339644],
       [0.015     , 0.02      , 0.15214968],
       [0.015     , 0.03      , 0.15213483],
       [0.015     , 0.04      , 0.15212054],
       [0.01      , 0.        , 0.19730903],
       [0.01      , 0.01      , 0.17449403],
       [0.01      , 0.02      , 0.15308178],
       [0.01      , 0.03      , 0.15213483],
       [0.01      , 0.04      , 0.15212054],
       [0.005     , 0.        , 0.20810852],
       [0.005     , 0.01      , 0.18541586],
       [0.005     , 0.02      , 0.16470214],
       [0.005     , 0.03      , 0.15213483],
       [0.005     , 0.04      , 0.15212054],
       [0.        , 0.        , 0.21873681],
       [0.        , 0.01      , 0.1962259 ],
       [0.

In [14]:
row = 0

for i in range(len(var_names)):
    for j in range(i+1, len(var_names)):
        # Get data from Normalized Data folder
        data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)
        regression_out.loc[row,["x","y"]] = [var_names[i], var_names[j]] # Set first two columns of DataFrame to names of variables

        # Remove data points where frequency change is stuck at ~0.15 Hz
        data = data[data[:,-1] > 0.155,:]

        # Process and perform regression on data
        x_data = data[:,:-1]
        y_data = data[:,-1].T
        model = model.fit(x_data, y_data)

        # Store regression output in regression_out DataFrame
        regression_out.loc[row, ["bx", "by", "bxy"]] = model["regression"].coef_
        regression_out.loc[row, "a"] = model["regression"].intercept_
        regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

        row += 1

regression_out

,x,y,bx,by,bxy,a,R^2
0,demand,trans,-1.957281,0.679595,-9.340901,0.210968,0.946422
1,demand,storage,-2.182994,-2.240103,-2.320674,0.219,0.999932
2,demand,inertia,-1.552965,-0.04764,0.483293,0.202473,0.707521
3,demand,solar,-2.156659,-2.147014,-7.854388,0.218538,0.999412
4,trans,storage,0.702222,-1.93833,-8.048787,0.209452,0.970831
5,trans,inertia,0.8246,-0.047754,-0.761481,0.193833,0.929022
6,trans,solar,0.682776,-1.628505,-7.777049,0.20799,0.954255
7,storage,inertia,-1.615138,-0.053258,-4.337232,0.20592,0.790517
8,storage,solar,-2.220928,-2.157349,-6.286821,0.218541,0.999592
9,inertia,solar,-0.050047,-1.453128,-0.4582,0.203512,0.759012


In [15]:
regression_out.to_csv("adjusted-regression-output.csv", index=False)

In [3]:
# Redefine regression_out to store regression coefficients for parameters when varied independently
regression_out = pd.DataFrame(index=pd.RangeIndex(stop=len(var_names)), columns=["x", "bx", "a", "R^2"])
regression_out["x"] = var_names
regression_out

,x,bx,a,R^2
0,demand,NaN,NaN,NaN
1,trans,NaN,NaN,NaN
2,storage,NaN,NaN,NaN
3,inertia,NaN,NaN,NaN
4,solar,NaN,NaN,NaN


In [4]:
model = LinearRegression()

In [5]:
row = 0

# Perform univariate regression on demand response
i = 0
j = 1
# Get data from Normalized Data folder
data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)

# Process and perform regression on data
x_data = data[::5,0].reshape(-1,1)
y_data = data[::5,-1].T
model = model.fit(x_data, y_data)

# Store regression output in regression_out DataFrame
regression_out.loc[row, "bx"] = model.coef_[0]
regression_out.loc[row, "a"] = model.intercept_
regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

row += 1


# Perform univariate regression on the other 4 parameters
for j in range(1, len(var_names)):
    # Get data from Normalized Data folder
    data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)

    # Process and perform regression on data
    if j == 4:
        x_data = data[-5:-1,1].reshape(-1,1)
        y_data = data[-5:-1,-1].T
    elif j == 2:
        x_data = data[-5:-2,1].reshape(-1,1)
        y_data = data[-5:-2,-1].T
    else:
        x_data = data[-5:,1].reshape(-1,1)
        y_data = data[-5:,-1].T
    model = model.fit(x_data, y_data)

    # Store regression output in regression_out DataFrame
    regression_out.loc[row, "bx"] = model.coef_[0]
    regression_out.loc[row, "a"] = model.intercept_
    regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

    row += 1

regression_out


,x,bx,a,R^2
0,demand,-2.176816,0.218907,0.999914
1,trans,0.744551,0.209213,0.93021
2,storage,-2.235755,0.21885,0.999951
3,inertia,-0.051099,0.204301,0.7394
4,solar,-2.202674,0.218756,0.99931


In [20]:
regression_out.to_csv("adjusted-indiv-regression-output.csv", index=False)